# Übung 7 - Recurrent Neural Networks

In dieser Übung wirst du ein RNN mittels Keras selbst erstellen und trainieren.

Das RNN soll Zeichenketten der Form `123+654` Zeichen für Zeichen one-hot kodiert als Eingabe erhalten und anschließend das Ergebnis der beschriebenen Rechnung zeichenweise ausgeben.
Es handelt sich hierbei also um *sequence to sequence learning*, da wir aus einer Eingabesequenz anschließend eine Ausgabesequenz erzeugen.

Die Trainingsdaten, auf denen wir das Netz trainieren, können wir selbst erzeugen.

In [ ]:
from keras.models import Sequential, Model
from keras.layers import LSTM, GRU, SimpleRNN, RepeatVector, TimeDistributed, Dense, Input, Lambda
import keras.backend as K
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
DIGITS = 3

## One-hot encoding

Zunächst benötigen wir eine Klasse, welche die one-hot Kodierung und Dekodierung übernimmt.

**Aufgabe**: Implementiere eine Klasse, welche zu Kodierung und Dekodierung der Eingabesequenzen verwendet werden kann. Diese soll folgende Funktionalitäten bieten:
* Übergabe das Alphabets als Zeichenkette bei der Objekterzeugung
* Kodierung eines Strings: Umwandlung eines Strings in eine Matrix, welche die Vektoren aus der one-hot kodierten Eingabe enthält. Zusätzlich soll eine Länge `length` angegeben werden, bis zu welcher Länge die Zeichenkette mit Leerzeichen aufgefüllt wird.
    * Dimensionen der entstehenden Matrix: `(length, alphabet_length)`
    * **Hinweis**: Dieses Auffüllen der Sequenz mit Leerzeichen erleichtert uns später die Erstellung des Modells. Natürlich eignen sich RNNs auch dazu Sequenzen unterschiedlicher Länge als Eingabe einzulesen, was diesen Schritt in anderen Fällen obsolet macht.
* Dekodierung eines Vektors. Als Eingabe erhält die Funktionen einen Vektor mit den Wahrscheinlichkeiten der Auswahl der Zeichen, also einen Vektor mit `alphabet_length` Einträgen. Hier reicht es aus das Zeichen, welches mit höchster Wahrscheinlichkeit ausgewählt wird, zu ermitteln und zurückzugeben.

## Erzeugung der Trainingsdaten

Zum Training des Netzes benötigen wir Trainingsdaten, welche wir uns in ausreichender Menge selbst erstellen können.

**Aufgabe**: Erstelle eine Funktion, welche Trainingsdaten einer bestimmten Größe für unsere Problemstellung erzeugt.
* Per Parameter kann diese eine Anzahl Ziffern übergeben bekommen, aus denen eine Zahl maximal bestehen darf.
* Beachte dabei: Diese Funktion gibt Zeichenketten, keine `integer` zurück
* Es sollte keine Aufgabe mehrfach in den Daten vorkommen

**Aufgabe**: Erzeuge einen Datensatz mit 50000 Einträgen, welchen wir für das Training benutzen und erzeuge die One-Hot kodierte Matrix dieses Datensatzes.
Teile diesen anschließend in 20% Validierungs- und 80% Trainingsdatensatz ein.

## Modell

Zur Lösung unseres Problems benötigen wir ein Sequence-to-Sequence Modell, also ein Modell, welches zunächst elementweise eine Eingabesequenz erhält und anschließend, ebenfalls elementweise, eine Ausgabesequenz ausgibt.
Das Einlesen geschieht im Encoder (im Bild weiß), das anschließende Auswerten des sich ergebenden Zellzustandes im Decoder (grau), welche beide RNN-Zellen sind.
Wir verwenden hier zunächst LSTM-Zellen.

![Sequence to Sequence Modell](seq2seq.png)

Wir bauen dieses Modell wie folgt mit Hilfe von `Sequential` in Keras auf.


In [ ]:
HIDDEN_SIZE = 256
ALPHABET_LENGTH = len('0123456789+ ')
MAXLEN = DIGITS * 2 + 1 # Maximallänge eines Eingabestrings

In [ ]:
model = Sequential()
model.add(LSTM(HIDDEN_SIZE, input_shape=(MAXLEN, ALPHABET_LENGTH))) # Encoder
model.add(RepeatVector(DIGITS + 1))  # Stellt dem RNN im nächsten Schritt die Ausgabe des vorherigen bereit
model.add(LSTM(HIDDEN_SIZE, return_sequences=True)) # Decoder
model.add(TimeDistributed(Dense(ALPHABET_LENGTH, activation='softmax'))) # Wendet eine `Dense` Schicht auf jede Ausgabe des Decoders an
                                # und ermittelt mittels 'softmax'-Funktion die Auswahlwahrscheinlichkeiten der Zeichen
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

## Training

**Aufgaben**: Trainiere das oben erstellte Netz für 50 Epochen. Denke daran einen Teil der Daten zur Validation zu verwenden.
* Implementiere die untenstehende Funktion `train`
* Um den Trainingsfortschritt zu beobachten soll nach jeder fünften Epoche eine Ausgabe erzeugt werden, in der zehn Einträge des **Valisierungs**datensatzes und die vom neuronalen Netz erzeugten Ausgabesequenzen visualisiert werden. Hierbei sollte auch sichtbar sein, ob die Ausgabe des Netzes korrekt war. Wer möchte kann dazu die untenstehende Klasse `colors` verwenden, um mit ANSI color codes die Ausgabe einzufärben.
* Führe `train` aus. Plotte den Verlauf von `loss` und `accuracy`, sowohl auf dem Trainings- als auch auf dem Validierungsset.

In [ ]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

In [ ]:
def train(model, X_train, y_train, X_val, y_val, encoder, epochs):
    loss, acc, val_loss, val_acc = [], [], [], []
    
    #TODO
    
    raise NotImplementedError
    return loss, acc, val_loss, val_acc

## Subtraktion statt Addition

**Aufgabe**: Wir werden nun das oben stehende Netz auf Subtraktion statt Addition trainieren.
Implementiere dazu eine Funktion oder ändere die obenstehende ab, um entsprechende Trainingsdaten zu erzeugen.

**Aufgabe**: Erzeuge wie oben einen Datensatz und One-Hot-Kodiere ihn.

**Aufgabe**: Trainiere sowohl ein "frisches", untrainiertes Netz so wie eine Kopie des zuvor auf die Addition trainierten Netzes wie oben beschrieben und vergleiche den Trainingserfolg. Was fällt auf?

## RNN, LSTM, GRU

**Aufgabe**: Ändere die zuvor verwendete Netzarchitektur so ab, dass anstelle von `LSTM`s `SimpleRNN`s oder `GRU`s verwendet werden, trainiere diese und vergleiche erneut den Lernerfolg. Was fällt auf?